In [ ]:
import random

num_nodes = 10
node_updates_per_pe = 1
number_pes = 1

PEs = {key: [] for key in range(1, number_pes+1)}

print(PEs)

In [ ]:
import random

def assign_values_to_pes(num_nodes, node_updates_per_pe, number_pes):
    # Create a pool of values to choose from
    values_pool = list(range(0, num_nodes))
    random.shuffle(values_pool)
    
    # Distribute values among PEs
    PEs = {}
    for i in range(number_pes):
        start_index = i * node_updates_per_pe
        end_index = min((i + 1) * node_updates_per_pe, num_nodes)
        pe_values = values_pool[start_index:end_index]
        PEs[i] = pe_values
    
    return PEs

# Example usage:
num_nodes = 5
node_updates_per_pe = 1
number_pes = 2

PEs = assign_values_to_pes(num_nodes, node_updates_per_pe, number_pes)
print(PEs)

In [ ]:
def check_value_in_previous_pes(PEs, pe_id, value):
    # Check if the value is in the current PE
    if value in PEs.get(pe_id, []):
        return True
    
    # Check if the value is in previous PEs
    for key in range(1, pe_id):
        if value in PEs[key]:
            return True
    
    return False

# Example usage:
PEs = {0: [1, 2, 3], 1: [4, 5, 6], 2: [7, 8, 9]}
pe_id = 1

value = 5

result = check_value_in_previous_pes(PEs, pe_id, value)
print(f"Is the value {value} in keys 1 to {pe_id}? {result}")


In [2]:
import numpy as np
from numba import njit

@njit(fastmath=True)
def check_value_in_previous_pes(PEs, pe_id, value, inc_current=False):
    """
    Check if a given value is present in the NumPy array for a specified processing element (PE).
    If inc_current is True, it also checks in the current PE.

    Args:
    PEs (dict): A dictionary where each key is a PE id and each value is a NumPy array of node indices.
    pe_id (int): The processing element id to check the value against.
    value (int): The value (node index) to check for.
    inc_current (bool): Whether to include the current PE's array in the check.

    Returns:
    bool: True if the value is found in the specified PE's array, False otherwise.
    """
    if pe_id in PEs:
        array = PEs[pe_id]  # Directly access the array if the key exists
        return value in array
    else:
        return False  # Return False if the key does not exist

# Example usage within other Numba-compiled code might look like this:
@njit
def some_function_using_check():
    PEs = {0: np.array([1, 2, 3], dtype=np.int64), 1: np.array([4, 5, 6], dtype=np.int64)}
    pe_id = 0
    value = 4
    result = check_value_in_previous_pes(PEs, pe_id, value, inc_current=True)
    print("Value found:", result)

some_function_using_check()

Value found: False
